
# Raman Spectra Analysis — Tutorial Notebook

**Goal.** This is a *tutorial-style* notebook (not a model solution) that demonstrates a clean, reusable workflow for analyzing Raman spectra. It includes small, well-documented helper functions and uses **synthetic example data** you can regenerate to test the full pipeline.

**What you'll learn / do here:**
- Create synthetic Raman spectra for testing (multiple peaks, baseline, noise).
- Pre-process spectra: cropping, cosmic-ray removal, smoothing, baseline correction, and normalization.
- Detect candidate peaks and estimate initial parameters.
- Fit peaks (Gaussian/Lorentzian/pseudo-Voigt) over selected regions.
- Compute peak metrics (position, height, FWHM, area) and export results.
- (Optional) Build a simple peak-ratio vs. concentration calibration using synthetic mixtures.

> **Note:** Keep this as a *helper toolbox* for your own analysis. Adapt function parameters to your instrument and samples. Real data will require careful tuning.



## 0. Setup

We use standard scientific Python libraries:
- `numpy`, `scipy`, `matplotlib`
- `pandas` for tidy result tables

> No internet access is required; install these locally if missing.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.signal import savgol_filter, find_peaks, medfilt
from scipy.optimize import curve_fit

# Matplotlib defaults (keep it simple; adjust as you like)
plt.rcParams["figure.figsize"] = (7, 4.5)
plt.rcParams["axes.grid"] = True

RNG = np.random.default_rng(42)  # for reproducibility



## 1. Synthetic Spectra Generator

To prototype the analysis without needing files, we'll synthesize spectra with:
- A wavenumber axis (e.g., 200–2000 cm⁻¹).
- Several peaks (Gaussian, Lorentzian, or pseudo-Voigt).
- A gentle baseline (polynomial) and additive noise.


In [ ]:

def gaussian(x, a, x0, sigma):
    """Gaussian peak; returns y."""
    return a * np.exp(-(x - x0)**2 / (2 * sigma**2))

def lorentzian(x, a, x0, gamma):
    """Lorentzian peak; returns y."""
    return a * (gamma**2 / ((x - x0)**2 + gamma**2))

def pseudo_voigt(x, a, x0, sigma, eta):
    """
    Pseudo-Voigt as a weighted sum of Gaussian and Lorentzian.
    - a: amplitude (peak height scale)
    - x0: center
    - sigma: Gaussian sigma
    - eta: mixing (0..1), 0=Gaussian, 1=Lorentzian
    """
    gauss = np.exp(-(x - x0)**2 / (2 * sigma**2))
    # Convert sigma -> gamma approx for Lorentz part
    gamma = sigma * np.sqrt(2*np.log(2))
    lor = (gamma**2) / ((x - x0)**2 + gamma**2)
    return a * (eta * lor + (1 - eta) * gauss)

def poly_baseline(x, coeffs):
    """Evaluate polynomial baseline with given coefficients (np.polyval order)."""
    return np.polyval(coeffs, x)

def make_spectrum(x, peaks, baseline_coeffs=(0,0,0), noise_std=0.0):
    """
    Build a synthetic spectrum.
    - x: wavenumber axis
    - peaks: list of dicts, each with:
        {'kind': 'gaussian'|'lorentzian'|'pvoigt',
         'a': amplitude, 'x0': center, 'w': width (sigma for gauss/pvoigt, gamma for lor),
         'eta': mix for pvoigt (0..1) }
    - baseline_coeffs: polynomial (highest first), e.g. (1e-7, -1e-3, 2)
    - noise_std: additive Gaussian noise stdev
    """
    y = np.zeros_like(x, dtype=float)
    for p in peaks:
        k = p.get('kind', 'gaussian').lower()
        a = p['a']
        x0 = p['x0']
        w = p['w']
        if k == 'gaussian':
            y += gaussian(x, a, x0, w)
        elif k == 'lorentzian':
            y += lorentzian(x, a, x0, w)
        elif k == 'pvoigt':
            eta = float(p.get('eta', 0.5))
            y += pseudo_voigt(x, a, x0, w, eta)
        else:
            raise ValueError(f"Unknown peak kind: {k}")
    y += poly_baseline(x, baseline_coeffs)
    if noise_std > 0:
        y += RNG.normal(0, noise_std, size=x.shape)
    return y


In [ ]:

# Example synthetic spectrum
x = np.linspace(200, 2000, 5000)  # Raman shift [cm^-1]
peaks = [
    {'kind': 'gaussian', 'a': 1.5, 'x0': 520, 'w': 8},
    {'kind': 'pvoigt',   'a': 1.0, 'x0': 1040, 'w': 10, 'eta': 0.3},
    {'kind': 'lorentzian','a': 0.7, 'x0': 1450, 'w': 12},
]
baseline_coeffs = (1e-8, -2e-4, 6)  # gentle quadratic baseline
y = make_spectrum(x, peaks, baseline_coeffs=baseline_coeffs, noise_std=0.03)

plt.plot(x, y)
plt.xlabel("Raman shift (cm$^{-1}$)")
plt.ylabel("Intensity (a.u.)")
plt.title("Synthetic Raman Spectrum (example)")
plt.show()



## 2. Pre-processing Helpers

Real Raman data often needs:
- **Cropping** to a region of interest (ROI)
- **Cosmic-ray spike removal** (simple median filter or Hampel-like approach)
- **Smoothing** with Savitzky–Golay (preserves peak shape)
- **Baseline correction** (polynomial or asymmetric least squares)
- **Normalization** (max, area, or vector)


In [ ]:

def crop(x, y, xmin=None, xmax=None):
    m = np.ones_like(x, dtype=bool)
    if xmin is not None:
        m &= x >= xmin
    if xmax is not None:
        m &= x <= xmax
    return x[m], y[m]

def remove_cosmic_rays(y, kernel_size=3, thresh=6.0):
    """
    Simple spike suppressor: compare to median-filtered signal.
    Replace outliers that exceed 'thresh' * local MAD (approx) with median value.
    """
    y_med = medfilt(y, kernel_size=kernel_size)
    resid = y - y_med
    mad = np.median(np.abs(resid - np.median(resid))) + 1e-12
    mask = np.abs(resid) > (thresh * 1.4826 * mad)
    y_fix = y.copy()
    y_fix[mask] = y_med[mask]
    return y_fix

def smooth_savgol(y, window_length=21, polyorder=3):
    if window_length % 2 == 0:
        window_length += 1  # must be odd
    return savgol_filter(y, window_length=window_length, polyorder=polyorder)

def baseline_polyfit(x, y, deg=2, mask_sigma=2.5, iters=3):
    """
    Robust polynomial baseline fit with iterative sigma-masking.
    Returns baseline estimate and (y - baseline).
    """
    mask = np.ones_like(y, dtype=bool)
    for _ in range(iters):
        coeffs = np.polyfit(x[mask], y[mask], deg)
        base = np.polyval(coeffs, x)
        resid = y - base
        s = np.std(resid[mask])
        mask = resid < (mask_sigma * s)
    coeffs = np.polyfit(x[mask], y[mask], deg)
    base = np.polyval(coeffs, x)
    return base, y - base

def normalize(y, method="max"):
    if method == "max":
        scale = np.max(y) if np.max(y) != 0 else 1.0
    elif method == "area":
        scale = np.trapz(np.abs(y)) or 1.0
    elif method == "vector":
        scale = np.linalg.norm(y) or 1.0
    else:
        raise ValueError("Unknown normalization method")
    return y / scale

def calibrate_axis(x_pixels, coeffs):
    """
    Map pixel/channel axis -> Raman shift via polynomial coeffs (highest power first).
    Placeholder to be used with real coefficients from 'calibration_hg_na.ipynb'.
    """
    return np.polyval(coeffs, x_pixels)


In [ ]:

# Demonstrate preprocessing on the example synthetic spectrum
x_roi, y_roi = crop(x, y, 350, 1700)
y_despike = remove_cosmic_rays(y_roi, kernel_size=5, thresh=5.0)
y_smooth  = smooth_savgol(y_despike, window_length=17, polyorder=3)
base, y_corr = baseline_polyfit(x_roi, y_smooth, deg=2, mask_sigma=2.0, iters=3)
y_norm = normalize(y_corr, method="max")

fig, ax = plt.subplots()
ax.plot(x_roi, y_roi, label="raw (ROI)")
ax.plot(x_roi, y_despike, label="despiked")
ax.plot(x_roi, y_smooth, label="smoothed")
ax.plot(x_roi, base, label="baseline (fit)")
ax.set_xlabel("Raman shift (cm$^{-1}$)")
ax.set_ylabel("Intensity (a.u.)")
ax.set_title("Preprocessing steps")
ax.legend()
plt.show()

plt.plot(x_roi, y_norm)
plt.xlabel("Raman shift (cm$^{-1}$)")
plt.ylabel("Normalized Intensity (a.u.)")
plt.title("Baseline-corrected & normalized")
plt.show()



## 3. Peak Detection & Initial Guesses

Use `scipy.signal.find_peaks` with thresholds on height, prominence, and minimum distance. Then estimate initial parameters for fitting.


In [ ]:

def estimate_sigma_from_width(x, peak_index, y, rel_height=0.5):
    """
    Rough sigma estimate from local width at rel_height (FWHM ~ 2.355*sigma for Gaussian).
    """
    peak_x = x[peak_index]
    peak_y = y[peak_index]
    target = peak_y * rel_height
    # left
    i = peak_index
    while i > 0 and y[i] > target:
        i -= 1
    left_x = x[i]
    # right
    j = peak_index
    while j < len(y)-1 and y[j] > target:
        j += 1
    right_x = x[j]
    fwhm = max(1e-9, right_x - left_x)
    sigma = fwhm / 2.355
    return max(1.0, sigma)  # keep a minimum width

def find_candidate_peaks(x, y, height=None, prominence=0.02, distance=30):
    peaks_idx, props = find_peaks(y, height=height, prominence=prominence, distance=distance)
    results = []
    prominences = props.get("prominences", np.zeros_like(peaks_idx, dtype=float))
    for k, idx in enumerate(peaks_idx):
        sigma0 = estimate_sigma_from_width(x, idx, y, rel_height=0.5)
        results.append({
            "index": int(idx),
            "x0": float(x[idx]),
            "height": float(y[idx]),
            "sigma0": float(sigma0),
            "prominence": float(prominences[k] if k < len(prominences) else 0.0),
        })
    return results


In [ ]:

cands = find_candidate_peaks(x_roi, y_norm, prominence=0.05, distance=40)
pd.DataFrame(cands)



## 4. Peak Fitting

Fit one or more peaks in a region. We'll implement Gaussian, Lorentzian, and pseudo-Voigt models with `scipy.optimize.curve_fit`.

> In practice, constrain parameters carefully and fit small windows around peaks of interest for robustness.


In [ ]:

def gaussian_sum(x, *params):
    """
    Sum of N Gaussians.
    params = [a1, x01, sigma1, a2, x02, sigma2, ...]
    """
    y = np.zeros_like(x, dtype=float)
    for i in range(0, len(params), 3):
        a, x0, s = params[i:i+3]
        y += gaussian(x, a, x0, s)
    return y

def lorentzian_sum(x, *params):
    y = np.zeros_like(x, dtype=float)
    for i in range(0, len(params), 3):
        a, x0, g = params[i:i+3]
        y += lorentzian(x, a, x0, g)
    return y

def pvoigt_sum(x, *params):
    """
    Sum of N pseudo-Voigts.
    params = [a1, x01, sigma1, eta1, a2, x02, sigma2, eta2, ...]
    """
    y = np.zeros_like(x, dtype=float)
    for i in range(0, len(params), 4):
        a, x0, s, eta = params[i:i+4]
        y += pseudo_voigt(x, a, x0, s, eta)
    return y

def fit_region(x, y, model="gaussian", x_min=None, x_max=None, init_peaks=None, bounds=None):
    """
    Fit peaks in [x_min, x_max] region.
    - model: 'gaussian'|'lorentzian'|'pvoigt'
    - init_peaks: list of dicts [{'a':..., 'x0':..., 'w':..., 'eta':...}]
    - bounds: tuple (lower, upper) arrays matching parameter length
    Returns popt, pcov, x_fit, y_fit
    """
    xm, ym = crop(x, y, x_min, x_max)
    if init_peaks is None or len(init_peaks) == 0:
        raise ValueError("Provide at least one initial peak guess in init_peaks.")

    if model == "gaussian":
        p0 = []
        for p in init_peaks:
            p0 += [p['a'], p['x0'], max(1.0, p['w'])]
        f = gaussian_sum
    elif model == "lorentzian":
        p0 = []
        for p in init_peaks:
            p0 += [p['a'], p['x0'], max(1.0, p['w'])]
        f = lorentzian_sum
    elif model == "pvoigt":
        p0 = []
        for p in init_peaks:
            p0 += [p['a'], p['x0'], max(1.0, p['w']), np.clip(p.get('eta', 0.5), 0.0, 1.0)]
        f = pvoigt_sum
    else:
        raise ValueError("Unknown model")

    popt, pcov = curve_fit(f, xm, ym, p0=p0, bounds=bounds if bounds is not None else (-np.inf, np.inf), maxfev=10000)
    yfit = f(xm, *popt)
    return popt, pcov, xm, yfit

def peak_metrics_gaussian(a, sigma):
    """
    For Gaussian: height = a, area = a * sigma * sqrt(2*pi), FWHM = 2.355*sigma
    """
    area = a * sigma * np.sqrt(2*np.pi)
    fwhm = 2.355 * sigma
    return {"height": a, "area": area, "fwhm": fwhm}


In [ ]:

# Example: Fit the ~1040 cm^-1 region with a single pseudo-Voigt
x_min, x_max = 990, 1090
init = [{"a": 0.9, "x0": 1040, "w": 8, "eta": 0.3}]  # rough guesses

popt, pcov, xf, yf = fit_region(x_roi, y_norm, model="pvoigt", x_min=x_min, x_max=x_max, init_peaks=init)

plt.plot(x_roi, y_norm, alpha=0.6, label="data")
plt.plot(xf, yf, label="fit")
plt.xlim(x_min, x_max)
plt.xlabel("Raman shift (cm$^{-1}$)")
plt.ylabel("Normalized Intensity (a.u.)")
plt.title("Fit around ~1040 cm$^{-1}$ (pseudo-Voigt)")
plt.legend()
plt.show()

popt



## 5. Simple Quantification Example (Synthetic Mixtures)

We'll generate mixtures of two components (A & B) with peaks around 880 and 1040 cm⁻¹.
We'll then compute a **peak area ratio** and see how it correlates with the mixture fraction.
This is purely illustrative.


In [ ]:
# --- Synthetic generator (mit optionalem internem Standard) + Fit-Helper + R² ---

def synth_mix_spectrum(x, frac_A, noise_std=0.003, baseline_coeffs=(0, 0, 0), include_is=True):
    """
    Linear gemischtes Spektrum aus A und B.
      - A: starker Peak @ ~880
      - B: starker Peak @ ~1040
      - Optionaler interner Standard (IS) @ ~1000 mit konstanter Intensität.
    Es gibt KEINE Normalisierung, damit die Flächen linear mit der "Konzentration" skalieren.
    """
    # Fixe, identische Breiten für saubere Flächenproportionalität
    wA, wB, wIS = 8.0, 9.0, 8.0

    peaks_A = [
        {'kind': 'gaussian',  'a': 1.20*frac_A,       'x0': 880.0,  'w': wA},
        {'kind': 'gaussian',  'a': 0.60*frac_A,       'x0': 1450.0, 'w': 10.0},
    ]
    peaks_B = [
        {'kind': 'pvoigt',    'a': 1.20*(1-frac_A),   'x0': 1040.0, 'w': wB, 'eta': 0.3},
        {'kind': 'lorentzian','a': 0.50*(1-frac_A),   'x0': 1300.0, 'w': 12.0},
    ]
    peaks_IS = []
    if include_is:
        peaks_IS = [{'kind': 'gaussian', 'a': 1.00, 'x0': 1000.0, 'w': wIS}]  # konstant!

    yA = make_spectrum(x, peaks_A, baseline_coeffs=(0, 0, 0), noise_std=0.0)
    yB = make_spectrum(x, peaks_B, baseline_coeffs=(0, 0, 0), noise_std=0.0)
    yIS = make_spectrum(x, peaks_IS, baseline_coeffs=(0, 0, 0), noise_std=0.0) if include_is else 0.0

    y = yA + yB + yIS + poly_baseline(x, baseline_coeffs)
    if noise_std and noise_std > 0:
        y += RNG.normal(0, noise_std, size=x.shape)
    return y

def area_from_gaussian_fit(x, y, center, half_window=15, a0=0.8, w0=8.0):
    """
    Fit eines einzelnen Gaussians im kleinen Fenster um 'center' und Ausgabe der Fläche.
    Achtung: für pseudo-Voigt/Lorentz brauchst du eigene Flächen-Formeln.
    """
    popt, pcov, xf, yf = fit_region(
        x, y, model="gaussian",
        x_min=center-half_window, x_max=center+half_window,
        init_peaks=[{"a": a0, "x0": center, "w": w0}],
        bounds=None
    )
    a, x0, sigma = popt[:3]
    return float(a * sigma * np.sqrt(2*np.pi))  # Gaussian area

def r2_score(y_true, y_pred):
    """Einfaches Bestimmtheitsmaß R²."""
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2) + 1e-15
    return 1.0 - ss_res/ss_tot


In [ ]:
# --- Calibration with internal standard (A/IS vs Fraction A) ---

fractions = np.linspace(0, 1, 9)
areas_A, areas_IS = [], []

for f in fractions:
    y = synth_mix_spectrum(x, frac_A=f, noise_std=0.003, baseline_coeffs=(0,0,0), include_is=True)
    x_r, y_r = crop(x, y, 350, 1700)

    A  = area_from_gaussian_fit(x_r, y_r, center=880.0,  half_window=15, a0=1.0, w0=8.0)
    IS = area_from_gaussian_fit(x_r, y_r, center=1000.0, half_window=15, a0=1.0, w0=8.0)

    areas_A.append(A)
    areas_IS.append(IS)

df_cal = pd.DataFrame({
    "fraction_A": fractions,
    "area_A_880": areas_A,
    "area_IS_1000": areas_IS,
    "ratio_A_over_IS": np.array(areas_A) / (np.array(areas_IS) + 1e-12),
})

# Lineare Kalibration
def linear(x, m, b): return m*x + b

y_ratio = df_cal["ratio_A_over_IS"].values
popt, _ = curve_fit(linear, df_cal["fraction_A"], y_ratio)
yfit = linear(df_cal["fraction_A"], *popt)
r2 = r2_score(y_ratio, yfit)

# Plot
plt.scatter(df_cal["fraction_A"], y_ratio, label="data")
xf = np.linspace(0, 1, 200)
plt.plot(xf, linear(xf, *popt), label=f"fit (R²={r2:.2f})")
plt.xlabel("Fraction A (synthetic)")
plt.ylabel("Area ratio A(880) / IS(1000)")
plt.title("Calibration: A/IS — linear with internal standard")
plt.legend()
plt.show()

print("Calibration (A/IS) — slope, intercept, R² = ", popt[0], popt[1], round(r2,2))


# Optional: Tabelle inspizieren
df_cal



## 6. Export Results

Export tables as CSV for further analysis or reporting.


In [ ]:

out_path = "calibration_example_results.csv"
df_cal.to_csv(out_path, index=False)
out_path



## 7. Next Steps & Tips

- Replace synthetic data with your real spectra (CSV or vendor export).
- If you have pixel/channel axis, apply your calibration (e.g., from `calibration_hg_na.ipynb`).
- For robust fits:
  - Work in small windows around peaks of interest.
  - Constrain parameters (bounds) and provide good initial guesses.
  - Use replicate spectra to estimate uncertainty.
- Consider more advanced baselines (e.g., asymmetric least squares) or deconvolution if peaks are overlapped.
- Document your chosen parameters in a lab notebook to keep analysis reproducible.
